In [1]:
import pandas as pd

In [2]:
df_train=pd.read_csv('.//Train/Train.csv')
df_test=pd.read_csv('.//Test/Test.csv')

In [3]:
df_train.head()

,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos


In [4]:
k=df_train.loc[:,'label']
print(k)

0        pos
1        pos
2        pos
3        pos
4        pos
        ... 
39995    neg
39996    neg
39997    neg
39998    pos
39999    pos
Name: label, Length: 40000, dtype: object


In [5]:
df_test.head()

,review
0,Remember those old kung fu movies we used to w...
1,This movie is another one on my List of Movies...
2,How in the world does a thing like this get in...
3,"""Queen of the Damned"" is one of the best vampi..."
4,The Caprica episode (S01E01) is well done as a...


In [6]:
test_data=df_test.values
print(test_data.shape) 
print(test_data[0:2])

(10000, 1)
[['Remember those old kung fu movies we used to watch on Friday and Saturday late nights when our babysitters THOUGHT we were in charge? Well, this movie plays exactly like one of those movies. Patsy Kensit\'s biggest claim to fame was the love interest to Mel Gibson\'s character in "Lethal Weapon 2," and this performance was one of the reasons why she\'s never made it big: she\'s a terrible actress.<br /><br />In "Lethal Weapon 2," I thought she was cute. Cute enough to check out some of the other movies she\'d been in, including "Loves Music, Loves to Dance" another big let down, which I, obviously, was not impressed with, either. But, as attractive as she is to my eyes, my soul screamed at me to turn it off because she played another cheap, predictable role, and done it very badly.<br /><br />In this movie, Kensit stars as a comedienne (and not a good one, either) who\'s working the clubs of France (couldn\'t cut it in her own homeland, so she\'s making THEIR ears bleed),

In [7]:
test_data=test_data.flatten()  #for linear shape
print(test_data.shape)
test_data[0:2]

(10000,)


array(['Remember those old kung fu movies we used to watch on Friday and Saturday late nights when our babysitters THOUGHT we were in charge? Well, this movie plays exactly like one of those movies. Patsy Kensit\'s biggest claim to fame was the love interest to Mel Gibson\'s character in "Lethal Weapon 2," and this performance was one of the reasons why she\'s never made it big: she\'s a terrible actress.<br /><br />In "Lethal Weapon 2," I thought she was cute. Cute enough to check out some of the other movies she\'d been in, including "Loves Music, Loves to Dance" another big let down, which I, obviously, was not impressed with, either. But, as attractive as she is to my eyes, my soul screamed at me to turn it off because she played another cheap, predictable role, and done it very badly.<br /><br />In this movie, Kensit stars as a comedienne (and not a good one, either) who\'s working the clubs of France (couldn\'t cut it in her own homeland, so she\'s making THEIR ears bleed), who\'

In [8]:
train_data=df_train.values

In [9]:
print(train_data[0:15,1]) #tags

['pos' 'pos' 'pos' 'pos' 'pos' 'pos' 'neg' 'neg' 'pos' 'pos' 'neg' 'pos'
 'pos' 'neg' 'pos']


In [10]:
print(train_data[:2,0])  #reviews

["mature intelligent and highly charged melodrama unbelivebly filmed in China in 1948. wei wei's stunning performance as the catylast in a love triangle is simply stunning if you have the oppurunity to see this magnificent film take it"
 'http://video.google.com/videoplay?docid=211772166650071408&hl=en Distribution was tried.<br /><br />We opted for mass appeal.<br /><br />We want the best possible viewing range so, we forgo profit and continue our manual labor jobs gladly to entertain you for working yours.<br /><br />View Texas tale, please write about it... If you like it or not, if you like Alex or not, if you like Stuie, Texas or Texas tale... Just write about it.<br /><br />Your opinion rules.']


In [11]:
train_data.shape

(40000, 2)

In [12]:
x_train=train_data[:,0]
x_train.shape

(40000,)

In [13]:
y_train=train_data[:,1]
y_train.shape

(40000,)

In [14]:
x_test=test_data
x_test.shape

(10000,)

In [16]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords





#initialize objects
tokenizer=RegexpTokenizer(r'\w+')  
                                   
en_stopwords=set(stopwords.words('english'))  #english langauge stopwords

ps=PorterStemmer()



def getCleanReview(review):
    review=review.lower()    
    review=review.replace('<br /><br />',' ')  
    
    #tokenize
    tokens=tokenizer.tokenize(review)
    new_tokens=[token for token in tokens if token not in en_stopwords or token == 'not' ]
    stemmed_tokens=[ps.stem(token) for token in new_tokens]
    
    cleaned_review= ' '.join(stemmed_tokens) 
    
    return cleaned_review


In [17]:
x_train_clean=[getCleanReview(i) for i in x_train ]
x_test_clean=[getCleanReview(i) for i in x_test]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
#vectorization
import time
t1=time.time()
cv=CountVectorizer(ngram_range=(1,3))
xtrain_vec=cv.fit_transform(x_train_clean)
t2=time.time()
print(t2-t1)




35.20689940452576


In [20]:
#vectorize test data
xtest_vec=cv.transform(x_test_clean)


In [21]:
print(xtest_vec.shape)

(10000, 6567532)


In [22]:
from sklearn.naive_bayes import BernoulliNB

In [23]:
bnb=BernoulliNB()   

In [24]:
#training
bnb.fit(xtrain_vec,y_train) 

BernoulliNB()

In [25]:
predictions=bnb.predict(xtest_vec)

In [26]:
print(type(predictions))
predictions.shape

<class 'numpy.ndarray'>


(10000,)

In [27]:
bnb.score(xtrain_vec,y_train) 

0.9995

In [28]:
#saving predictions in a csv file
df=pd.DataFrame(data=predictions, columns=['target']) .reset_index()
df.columns = ['Id',"label"]
df.to_csv('predictions.csv',index=False) #gave 86% accuracy on test data